## Importing Libraries

In [1]:
import numpy as np
import imageio as iio
import matplotlib.pyplot as plt

## Method 1

### Fourier Transform of Image

In [12]:
img = iio.imread('q1.jpg')

# calculate rgb
r = img[:,:,0]
g = img[:,:,1]
b = img[:,:,2]

# calculate fourier transform of channels
fr = np.fft.fftshift(np.fft.fft2(r))
fg = np.fft.fftshift(np.fft.fft2(g))
fb = np.fft.fftshift(np.fft.fft2(b))

# calculate fourier transform of image
fimg = np.zeros((img.shape[0], img.shape[1], img.shape[2]), dtype='complex')
fimg[:,:,0] = fr.copy()
fimg[:,:,1] = fg.copy()
fimg[:,:,2] = fb.copy()

iio.imwrite('q1_res01.jpg', np.array(20*np.log10(np.abs(fimg)), dtype='uint8'))

### Build Highpass Filter

In [13]:
filt = np.zeros((img.shape[0], img.shape[1], img.shape[2]))
sigma = 70
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        for k in range(img.shape[2]):
            filt[i,j,k] = np.exp(-((i-0.5*img.shape[0])**2+(j-0.5*img.shape[1])**2)/sigma**2)
            
filt = 1-filt/np.max(filt)
iio.imwrite('q1_res02.jpg', np.array(filt*255, dtype='uint8'))

### Filtering Image

In [16]:
k = 0.8
fnewimg = (1 + k * filt) * fimg
iio.imwrite('q1_res03.jpg', np.array(20*np.log10(np.abs(fnewimg)), dtype='uint8'))

In [20]:
newr = np.fft.ifft2(np.fft.ifftshift(fnewimg[:,:,0]))
newg = np.fft.ifft2(np.fft.ifftshift(fnewimg[:,:,1]))
newb = np.fft.ifft2(np.fft.ifftshift(fnewimg[:,:,2]))
new_img = np.zeros((img.shape[0], img.shape[1], img.shape[2]), dtype='complex')
new_img[:,:,0] = newr.copy()
new_img[:,:,1] = newg.copy()
new_img[:,:,2] = newb.copy()

new_img = 255*np.abs((new_img-np.min(new_img))/(np.max(new_img)-np.min(new_img)))

iio.imwrite('q1_res04.jpg', np.array(new_img, dtype='uint8'))

## Method 2

### Unsharp Mask

In [6]:
img = iio.imread('q1.jpg')

# calculate rgb
r = img[:,:,0]
g = img[:,:,1]
b = img[:,:,2]

# calculate fourier transform of channels
fr = np.fft.fftshift(np.fft.fft2(r))
fg = np.fft.fftshift(np.fft.fft2(g))
fb = np.fft.fftshift(np.fft.fft2(b))

fnewr = np.zeros((img.shape[0], img.shape[1]), dtype='complex')
fnewg = np.zeros((img.shape[0], img.shape[1]), dtype='complex')
fnewb = np.zeros((img.shape[0], img.shape[1]), dtype='complex')

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        fnewr[i,j] = 4*np.pi**2*((i-img.shape[0]//2)**2+(j-img.shape[1]//2)**2)*fr[i,j]
        fnewg[i,j] = 4*np.pi**2*((i-img.shape[0]//2)**2+(j-img.shape[1]//2)**2)*fg[i,j]
        fnewb[i,j] = 4*np.pi**2*((i-img.shape[0]//2)**2+(j-img.shape[1]//2)**2)*fb[i,j]
    

fnew = np.zeros((img.shape[0], img.shape[1], img.shape[2]), dtype='complex')
fnew[:,:,0] = fnewr.copy()
fnew[:,:,1] = fnewg.copy()
fnew[:,:,2] = fnewb.copy()
        
iio.imwrite('q1_res05.jpg', np.array(20*np.log10(np.abs(fnew)+1e-3), dtype='uint8'))

mask = np.zeros((img.shape[0], img.shape[1], img.shape[2]), dtype='complex')
mask[:,:,0] = np.fft.ifft2(np.fft.ifftshift(fnewr)).copy()
mask[:,:,1] = np.fft.ifft2(np.fft.ifftshift(fnewg)).copy()
mask[:,:,2] = np.fft.ifft2(np.fft.ifftshift(fnewb)).copy()
mask_img = np.uint8(np.abs((mask-np.min(mask))/(np.max(mask)-np.min(mask))*255))
iio.imwrite('q1_res06.jpg', mask_img)

### Create Final Image

In [7]:
k = 1e-7
new_img = img+k*mask
new_img = 255*np.abs((new_img-np.min(new_img))/(np.max(new_img)-np.min(new_img)))
iio.imwrite('q1_res07.jpg', np.array(new_img, dtype='uint8'))